In [ ]:
myAPIkey = "ADD YOUR OWN API KEY HERE!"

In [ ]:
import requests, json

def getNYTArticles(year, month, apiKey):
    """Function that sends a request to the NYT API for all articles in a month
    and then stores the results in a JSON file.
    """
    # create URL
    URL = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={apiKey}"

    # send the request to get the data
    data = requests.get(URL)
    if data.status_code == 200:
        print("Successfully got the data.")

    dataJson = data.json() # get response as JSON

    with open(f"NYT_{year}-{month}.json", 'w') as fout:
        json.dump(dataJson, fout)

In [ ]:
getNYTArticles(2024, 2, myAPIkey)

In [ ]:
with open("NYT_2024-2.json") as fin:
    articles = json.load(fin)

print(type(articles))

In [ ]:
articles.keys()

In [ ]:
for key in articles:
    print(key, type(articles[key]))

In [ ]:
articles['copyright']

In [ ]:
articles['response'].keys()

In [ ]:
for key in articles['response']:
    print(key, type(articles['response'][key]))

In [ ]:
articles['response']['meta']

In [ ]:
len(articles['response']['docs'])

In [ ]:
articles['response']['docs'][0]

In [ ]:
oneArticle = articles['response']['docs'][0]
for key in oneArticle:
    print(key, type(oneArticle[key]))

In [ ]:
sections = [article['section_name'] for article in articles['response']['docs']]

# Let's look up a few of them
sections[:5]

In [ ]:
from collections import Counter

distDct = Counter(sections) # count the occurrences of each section name

distDct.most_common(10)

In [ ]:
worked on with Maya from my group 

In [ ]:
def filterArticleDate(date, myAPIkey):
   year =  year = int(date[:4])
   day = int(date[8:])
   month = int(date[5:7]) if date[5] == '0' else int(date[5:7])
   file_name = f"{date}{month}.csv"
   data= requests.get(file_name) #200 is ok, 404 is not found
   if (data.status_code == 404): #csv not found, not in folder
      articles = getNYTArticles(year, month, myAPIkey) #get the articles
      csvFile = flattenAllArticles(date) #converts to csvFile
      df = pd.read_csv(csvFile) #turns to dataframe
      articlesOnDay = getArticlesOfDay(date) #get articles for desired date


In [ ]:
import requests, json
import pandas as pd
from collections import Counter
import os
   if os.path.exists(file_name):  # Check if CSV file exists
        df = pd.read_csv(file_name)  # Load CSV into dataframe
   else: 
        articles = getNYTArticles(year, month, myAPIkey)  # Get the articles
        csvFile = flattenAllArticles(date)  # Converts articles to CSV file
        df = pd.read_csv(csvFile)  # Load CSV into dataframe   
   articlesOnDay = df[df['date'] == date]  # Filter articles for desired date
   return articlesOnDay

desired_date = "2024-03-14"
articles = filterArticleDate(desired_date, myAPIkey)
print(articles)

In [ ]:
import requests, json
import pandas as pd
from collections import Counter

myAPIkey = 'CYJUbGC4TcnBoLg8PF1HlnEhJEEPHlBv' #maya 

'''
def get_NYT_articles(start_year, end_year):
    for year in range(start_year, end_year):
        for month in range(1, 13):
            URL = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={myAPIkey}"
            print(URL)

            data = requests.get(URL)
            data.status_code

'''
def getNYTArticles(year, month, apiKey):
    """Function that sends a request to the NYT API for all articles in a month
    and then stores the results in a JSON file.
    """
    # create URL
    URL = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={apiKey}"

    # send the request to get the data
    data = requests.get(URL)
    if data.status_code == 200:
        print("Successfully got the data.")

    dataJson = data.json() # get response as JSON

    with open(f"NYT_{year}-{month}.json", 'w') as fout:
        json.dump(dataJson, fout)

def getArticlesOfDay(date):
    year = int(date[:4])
    print(year)
    month = int(date[5:7]) if date[5] == '0' else int(date[5:7])
    print(month)
    getNYTArticles(year, month, myAPIkey)

    with open(f"NYT_{year}-{month}.json") as fin:
        data = json.load(fin)

    articles = data['response']['docs']

    articles_day = []
    for article in articles:
        if article['pub_date'][:10] == date:
            articles_day.append(article)
    return articles_day

def flattenArticle(article):
    output_dict = {}
    things_to_get = ['abstract', 'lead_paragraph', 'headline', 'keywords', 'pub_date', 'document_type', 'section_name']
    for info in things_to_get:
        output_dict[info] = article[info]
    keywords = output_dict['keywords']
    keywords = [tag['value'] for tag in keywords] 
    output_dict['keywords'] = keywords
    return output_dict

def flattenAllArticles(date):
    articles_of_the_day = getArticlesOfDay(date)
    output_list = []
    for article in articles_of_the_day:
        output_list.append(flattenArticle(article))
    
    df = pd.DataFrame(output_list)
    file_name = f"articles_{date[:7]}.csv"
    df.to_csv(file_name, index=False)
    return df
